In [30]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install scikit-learn
!pip install sentencepiece
!pip install sacremoses
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/jayjoshi/.zshenv:3: unmatched "
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/jayjoshi/.zshenv:3: unmatched "
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environme

In [11]:
import evaluate

metric_accuracy = evaluate.load("accuracy")

metric_glue = evaluate.load("glue", "mrpc")

metric_precision = evaluate.load("precision")

metric_recall = evaluate.load("recall")

metric_rocauc = evaluate.load("roc_auc")


score_dict = {"accuracy": "none", "glue": "none", "precision": "none", "recall": "none", "roc_auc": "none"}


def compute_metrics(labels):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
    score_dict["accuracy"] = metric_accuracy.compute(predictions=predictions, references=labels)
    score_dict["glue"] = metric_glue.compute(predictions=predictions, references=labels)
    score_dict["precision"] = metric_precision.compute(predictions=predictions, references=labels)
    score_dict["recall"] = metric_recall.compute(predictions=predictions, references=labels)
    score_dict["rocauc"] = metric_rocauc.compute(predictions=predictions, references=labels)



In [4]:
import pandas as pd

ds = pd.read_csv("CleanData/all_data_clean.csv")

# x = ds["Language"] == "Spanish"
# y = ds["Language"] == "English"
# z = ds["Language"] == "Hindi"

# ds = ds.where(x | y | z)

# condition = ds["Language"] != "English"

# ds = ds.where(condition)
ds.dropna(inplace=True)



texts = ds["Text"].tolist()
labels =[int(i) for i in  ds["Label"].tolist()]
languages = ds["Language"].tolist()


In [5]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', local_files_only=True)

In [7]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [8]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [17]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

import numpy as np

training_args = TrainingArguments(
    output_dir='./results/cleanData',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs/cleandata/',            # directory for storing logs
    logging_steps=10,
    # use_mps_device=True
)

model = AutoModelForSequenceClassification.from_pretrained("FineTuned_Model/xlm_r/")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    
)

predictions = trainer.predict(val_dataset)

preds = np.argmax(predictions.predictions, axis=-1)



    
score_dict


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file FineTuned_Model/xlm_r/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "FineTuned_Model/xlm_r/",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "s

KeyboardInterrupt: 

In [27]:
def compute_metrics(labels):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
    score_dict["accuracy"] = metric_accuracy.compute(predictions=preds, references=labels)
    score_dict["glue"] = metric_glue.compute(predictions=preds, references=labels)
    score_dict["precision"] = metric_precision.compute(predictions=preds, references=labels)
    score_dict["recall"] = metric_recall.compute(predictions=preds, references=labels)
#     score_dict["rocauc"] = metric_rocauc.compute(predictions=preds, references=labels)

In [28]:
# compute_metrics(predictions.label_ids)
# print(predictions[0])
print(predictions.label_ids)
print(preds)

compute_metrics(predictions.label_ids)


[1 0 1 ... 1 0 1]
[0 0 1 ... 0 0 0]


In [29]:
score_dict

{'accuracy': {'accuracy': 0.6674194519722975},
 'glue': {'accuracy': 0.6674194519722975, 'f1': 0.613608535945426},
 'precision': {'precision': 0.8917132689374683},
 'recall': {'recall': 0.46773333333333333},
 'roc_auc': 'none'}

In [37]:
(preds == 1).sum()

1967

In [38]:
(preds == 0).sum()

4675